In [83]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [11]:
df1 = pd.read_csv("data.csv", header=None, names=['Timestamp', 'x', 'y', 'z'])
display(df1.head())
display(df1.info())

,Timestamp,x,y,z
0,20240812172100,0.002197,-0.015137,1.023682
1,20240812172103,0.001709,-0.016968,1.022949
2,20240812172107,0.001953,-0.017090,1.023071
3,20240812172112,-0.003662,-0.018799,1.022949
4,20240812172117,0.001953,-0.016724,1.023560


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Timestamp  1020 non-null   int64  
 1   x          1020 non-null   float64
 2   y          1020 non-null   float64
 3   z          1020 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 32.0 KB


None

In [12]:
# Convert `Timestamp` column to time dtype
df1['Timestamp'] = pd.to_datetime(df1['Timestamp'], format='%Y%m%d%H%M%S')
df1.head()

,Timestamp,x,y,z
0,2024-08-12 17:21:00,0.002197,-0.015137,1.023682
1,2024-08-12 17:21:03,0.001709,-0.016968,1.022949
2,2024-08-12 17:21:07,0.001953,-0.017090,1.023071
3,2024-08-12 17:21:12,-0.003662,-0.018799,1.022949
4,2024-08-12 17:21:17,0.001953,-0.016724,1.023560


In [111]:
# Initialize the app
app = Dash()

# App layout
app.layout = html.Div([
    html.H1(children='Gyroscope data'),
    html.Hr(),
    
    # Drop down for graphs
    html.Label("Select Graph Type:"),
    dcc.Dropdown(
        id='graph-type',
        options=[
            {'label': 'Scatter Plot', 'value': 'scatter'},
            {'label': 'Line Chart', 'value': 'line'},
            {'label': 'Distribution Plot', 'value': 'distribution'},
        ],
        value = 'scatter'
    ),
    
    # Data Table
    dash_table.DataTable(data=df1.to_dict('records'), page_size=6),
    
    # Drop down for rotation selection
    dcc.Dropdown(id='rotation-type', 
                 options=[ {'label': 'x', 'value': 'x'}, 
                          {'label': 'y', 'value': 'y'}, 
                          {'label': 'z', 'value': 'z'} ], 
                 value=['x', 'y', 'z'], multi=True),
    
    # Input for the number of samples
    html.Label("Number of Samples: "),
    dcc.Input(id='num-samples', type='number', value=1020, min=100, step=20),
    
    # Container for the graph and summary table
    html.Div([
        dcc.Graph(figure={}, id='graph', style={'display': 'inline-block', 'width': '49%'}),
        dcc.Graph(id='data-summary', style={'display': 'inline-block', 'width': '49%', 'vertical-align': 'top'}),
    ], style={'display': 'flex', 'flex-wrap': 'wrap'})
])

@callback(
    Output("graph", "figure"), 
    Output('data-summary', 'figure'),
    Input("rotation-type", "value"),
    Input('graph-type', 'value'),
    Input('num-samples', 'value'),
)
def update(rotation, graph_type, num_sample):
    df2 = df1.head(num_sample)
    
    # Create plot based on selected graph
    fig = {}
    if graph_type == 'scatter':
        fig = px.scatter(df2, x='Timestamp', y=rotation)
    elif graph_type == 'line':
        fig = px.line(df2, x='Timestamp', y=rotation)
    elif graph_type == 'distribution':
        fig = px.histogram(df2, x=rotation)
        
    # Data summary table
    try:
        df3 = df2[rotation]
        summary = df3.describe().reset_index()
        summary_table = go.Figure(data=[go.Table(
            header=dict(values=list(summary.columns),
                        fill_color='paleturquoise',
                        align='left'),
            cells=dict(values=[summary[col] for col in summary.columns],
                    fill_color='lavender',
                    align='left')
            
        )])
    except Exception as e:
        summary_table = go.Figure(data=[go.Table(
            header=dict(values=["No data selected"],
                        fill_color='paleturquoise',
                        align='left'),
            cells=dict(values=[["Please select at least one rotation axis"]],
                       fill_color='lavender',
                       align='left')
        )])
    
    return fig, summary_table

# Run the app
if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>